In [1]:
import pandas as pd
import numpy as np

In [2]:

def auroc(prob,label):
    y_true=label#.data.numpy().flatten()
    y_scores=prob#.data.numpy().flatten()
    fpr,tpr,thresholds=roc_curve(y_true,y_scores)
    auroc_score=auc(fpr,tpr)
    return auroc_score,fpr,tpr

def auprc(prob,label):
    y_true=label#.data.numpy().flatten()
    y_scores=prob#.data.numpy().flatten()
    precision,recall,thresholds=precision_recall_curve(y_true,y_scores)
    auprc_score=auc(recall,precision)
    return auprc_score,precision,recall

from collections import Iterable


def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x


def prediction(predlabel, labels):
    predlabel_s = []
    labels_s = []
    for x in flatten(predlabel):
        predlabel_s.append(x)
    for x in flatten(labels):
        labels_s.append(x)

    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    acc = accuracy_score(labels_s, predlabel_s)
    precision = precision_score(labels_s, predlabel_s)
    recall = recall_score(labels_s, predlabel_s)
    f1 = f1_score(labels_s, predlabel_s)
    return acc, precision, recall, f1


/var/folders/37/2brzhtcs52160y_wjzcp3k_h0000gn/T/ipykernel_16168/2040278464.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [3]:
df=pd.read_csv('./hidden_emb.csv',header=None)

In [5]:
positive=pd.read_csv('../data/num_edge.txt',sep=' ',header=None).iloc[:1264,:]


In [6]:
a=list(zip(positive[0],positive[1]))

In [7]:
def generate_negative(genes, diseases, pos_samples):
    """ generate negative sample by random change one of a pairs"""
    pairs = []
    genes, diseases = list(genes), list(diseases)
    '''
    for ps in pos_samples:
        for k in range(0, nega_weight):
            index = random.randint(0, len(genes) - 1)
            while True:
                if (genes[index],ps[1]) not in all_pos_samples:
                    break
                index = random.randint(0, len(genes) - 1)
            pairs.add((genes[index], ps[1]))
    '''
    for i in genes:
        for j in diseases:
            if (i,j) not in pos_samples:
                pairs.append((i,j))
    print("len(ne_pairs)",len(pairs)) 
    return list(set(pairs))

In [8]:
neg=generate_negative(set(positive[0]), set(positive[1]), a)

len(ne_pairs) 100511


In [9]:

def merge_samples(p_samples, n_samples):

    pos_sample=[]
    neg_sample=[]
    for w in p_samples:
        pos_sample.append(w)

    for i in n_samples:
        neg_sample.append(i)
    import random
    neg_sample=random.sample(neg_sample,len(pos_sample))
    samples=pos_sample+neg_sample
    print(len(pos_sample),len(neg_sample))

    return samples


In [10]:
all_sample=merge_samples(a, neg)
allsample=pd.DataFrame(all_sample)
df=df.reset_index()
m1=pd.merge(allsample,df,left_on=0,right_on='index',how='left')
m2=pd.merge(m1,df,left_on='1_x',right_on='index',how='left')
emb=m2.iloc[:,5:]
emb['label']=1
emb.loc[1265:,'label']=0

1264 1264


In [11]:
def spe(Y_test,Y_pred,n):
    
    spe = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        number = np.sum(con_mat[:,:])
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        fp = np.sum(con_mat[:,i]) - tp
        tn = number - tp - fn - fp
        spe1 = tn / (tn + fp)
        spe.append(spe1)
    
    return spe


In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import precision_recall_curve,roc_curve,auc,balanced_accuracy_score,confusion_matrix
import sklearn
import sklearn.svm as svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import AdaBoostClassifier
aucs=[]
auprs=[]
accs=[]
precs=[]
res=[]
f1s=[]

fpr_s=[]
tpr_s=[]
precision_s=[]
recall_s=[]
spes=[]

mccs=[]
model = RandomForestClassifier()
#model = XGBClassifier()  
#model = svm.SVC(kernel='linear',probability=True)
#model = LogisticRegression()

#model=GaussianNB()
#model=AdaBoostClassifier()
# 对其中1折进行预测，对其他折进行训练
skfolds = StratifiedKFold(n_splits=5, shuffle=True,random_state=24)
X_train=emb.drop('label',axis=1).values
print(X_train.shape)
y_train_5=emb[['label']].values
for train_index, test_index in skfolds.split(emb.drop('label',axis=1), emb[['label']]):
    print(train_index)
    X_train_folds = X_train[train_index]
    y_train_folds = (y_train_5[train_index])
    X_test_fold = X_train[test_index]
    y_test_fold = (y_train_5[test_index])
    print(X_train_folds.shape)
    model.fit(X_train_folds, y_train_folds)

#     import pickle 

#     #1.保存成Python支持的文件格式Pickle
#     #在当前目录下可以看到new_app_model_v1.pickle
#     with open('RF_model_v1.pickle','wb') as fw:
#         pickle.dump(model,fw)



    y_prob=model.predict_proba(X_test_fold)[:,1]
    y_pred = model.predict(X_test_fold) 

    test_auc, fpr, tpr = auroc(y_prob, y_test_fold)
    test_aupr, precision, recall = auprc(y_prob, y_test_fold)
    acc, prec, re, f1 = prediction(y_pred, y_test_fold)   
    mcc=sklearn.metrics.matthews_corrcoef(y_test_fold, y_pred)
    c=spe(y_pred, y_test_fold,n=2)
    aucs.append(test_auc)
    auprs.append(test_aupr)
    accs.append(acc)
    precs.append(prec)
    res.append(re)
    f1s.append(f1)
    mccs.append(mcc)
    spes.append(c[1])
    fpr_s.append(fpr)
    tpr_s.append(tpr)
    precision_s.append(precision)
    recall_s.append(recall)

    print(test_auc,test_aupr,acc, prec, re, f1,mcc)

(2528, 256)
[   0    1    2 ... 2525 2526 2527]
(2022, 256)


/var/folders/37/2brzhtcs52160y_wjzcp3k_h0000gn/T/ipykernel_16168/4121847456.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_folds, y_train_folds)


0.8614257370057337 0.8566965057133827 0.8063241106719368 0.7859778597785978 0.841897233201581 0.8129770992366413 0.6142046804569072
[   0    3    4 ... 2522 2523 2524]
(2022, 256)


/var/folders/37/2brzhtcs52160y_wjzcp3k_h0000gn/T/ipykernel_16168/4121847456.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_folds, y_train_folds)


0.8875001952850381 0.8842695101214674 0.8003952569169961 0.7923076923076923 0.8142292490118577 0.8031189083820662 0.6010206037265402
[   0    1    2 ... 2525 2526 2527]
(2022, 256)


...

0.8483572622599946 0.8544311123868455 0.7806324110671937 0.77734375 0.7865612648221344 0.781925343811395 0.5613042846798874
[   0    1    2 ... 2525 2526 2527]
(2023, 256)


/var/folders/37/2brzhtcs52160y_wjzcp3k_h0000gn/T/ipykernel_16168/4121847456.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_folds, y_train_folds)


0.8497710019449151 0.8497807622382714 0.7821782178217822 0.7894736842105263 0.7707509881422925 0.78 0.5645346165949426
[   1    2    5 ... 2525 2526 2527]
(2023, 256)


/var/folders/37/2brzhtcs52160y_wjzcp3k_h0000gn/T/ipykernel_16168/4121847456.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_folds, y_train_folds)


0.8632285588807328 0.8534723666878176 0.803960396039604 0.7938931297709924 0.8221343873517787 0.8077669902912621 0.6082781570969439


In [14]:
np.average(aucs),np.average(auprs),np.average(accs),np.average(precs),np.average(res),np.average(f1s)

(0.8620565510752829,
 0.859730051429557,
 0.7946980785035025,
 0.7877992232135618,
 0.8071146245059289,
 0.797157668344273)

In [49]:
pd.DataFrame({'aucs':aucs,'auprs':auprs,'assc':accs,'prec':precs,'re':res,'f1':f1s,'mcc':mccs,'spec':spes}).to_csv('./XGB.csv',index=0)
pd.DataFrame(fpr_s).to_csv('fprs_xgb.csv',index=0,header=0)
pd.DataFrame(tpr_s).to_csv('tpr_xbg.csv',index=0,header=0)
pd.DataFrame(precision_s).to_csv('precision_xgb.csv',index=0,header=0)
pd.DataFrame(recall_s).to_csv('recall_xgb.csv',index=0,header=0)